# Assignment 3 - Graphs

In [2]:
import numpy as np

### Task 1: BFS to find shortest path

For this task we will use the Breadth First Search algorithm to find the shortest path in a directed, unweighted, acyclic graph. The graph is represented by a dictionary where the keys are the nodes and the values are the edges.

> Example: ``"a": ["b", "c"]`` means that the node "a" has edges to "b" and "c". To get these edges you can do:
> ```python
> node = "a"
> edges = graph[node]
> ```

The graph below is the one from the assignment description.

In [3]:
graph: dict = {
    "a": ["b", "d"],
    "b": ["c"],
    "c": ["f"],
    "d": ["e"],
    "e": ["b", "f"],
    "f": [],
}

Your task is to implement the `BFS()` function described below. The function takes the graph, a start node and an end node as input and returns the shortest path between the two nodes as a list.

> Tip: In order to find the shortest path to the end node you need to keep track of the shortest path to all the nodes you discover during your search. This can also be done in a dictionary. As an example below you see that the shortest path to the start node is only itself as the path to it from itself is the shortest path. You will also need to keep track of which nodes are visited, and which nodes you should visit next. 
> ```python
> shortest_paths = {
>     start_node: [start_node],
> }
> ```

In [4]:
def BFS(graph: dict, start_node: str, end_node: str) -> list:
    shortest_paths = {
        start_node: [start_node],
    }
    # nodes already checked
    visited = []
    # nodes "discovered" through the graph, yet to be checked
    queue = [start_node]

    # Runs while queue is not empty
    while queue:
        # Collects the next node(s) from current node
        next_nodes = graph[queue[0]]

        # Iterate through nodes that connect to current node
        for node in next_nodes:
            queue.append(node)

            #If node has not been visited, the path is saved
            if node not in visited:
                path = shortest_paths[queue[0]].copy() # Copy to make sure lists from memory is used
                path.append(node)
                shortest_paths[node] = path
                visited.append(node)
                    
        queue.pop(0)

    return shortest_paths[end_node]


# Testing the function
shortest_path = BFS(graph, "b", "f")
print(shortest_path)

['b', 'c', 'f']


### Task 2: Algorithm to find all topological orders

In this task you will implement an algorithm to find all topological orders of a directed, unweighted, acyclic graph. The graph is represented the same as in the previous task. 

In [5]:
graph: dict = {
    "a": ["c", "d"],
    "b": ["d", "e"],
    "c": ["f"],
    "d": [],
    "e": [],
    "f": ["e"],
}

The algorithm works as follows:

- We keep track of the number of incoming edges for each node.
- We iterate over all the nodes in the graph, 
  - recursively call the function

To start, first implement a function that counts the number of incoming edges for each node. The function should return a dictionary where the keys are the nodes and the values are the number of incoming edges.

> Example: 
> ``{'a': 0, 'b': 2, 'c': 1}``

In [6]:
def find_incoming_edges(graph: dict) -> dict:
    incoming_edges = {}

    #initiate incoming_edges
    for key in graph.keys():
        incoming_edges[key] = 0

    for node in graph:
        for child in graph[node]:
            incoming_edges[child] += 1

    return incoming_edges


# Testing the function
incoming_edges = find_incoming_edges(graph)
print(incoming_edges)

{'a': 0, 'b': 0, 'c': 1, 'd': 2, 'e': 2, 'f': 1}


Next you have to implement the recursive function that will find all topological orders. To guide you, parts of the function are already implemented, your job is to implement what is described in the "TODO" comments. The result of the function should a list of strings, where each string is a topological order of the graph.

> Example: ``["abcde", "abedc", "abdec"]`` for a graph with 3 topological orders.

In [7]:
def find_all_topological_orders(
    graph: dict,
    incoming_edges: dict = incoming_edges,
    visited: list = [],
    path: list = [],
) -> list:

    topological_orders = []

    for node in graph.keys():

        # TODO We only want to check nodes that dont have incoming edges,
        # and haven't already been visited
        if incoming_edges[node] != 0 or node in visited:
            continue

        # TODO "remove" the edges coming from the visited node (decrease incoming_edge count for the edges)
        # and add the node to the path while setting it as visited
        path.append(node)
        visited.append(node)

        for child in graph[node]:
            incoming_edges[child] -= 1

        # Recursively do this with the graph that now has the node "removed"
        topological_orders.extend(
            find_all_topological_orders(
                graph,
                incoming_edges,
                visited,
                path,
            )
        )

        # TODO backtrack:
        # We want to reset the changes we made above the recursive call so we can check other options

        visited.remove(node)
        path.pop()

        for child in graph[node]:
            incoming_edges[child] += 1

    # If the path includes all nodes in the graph, we have a valid topological order
    if len(path) == len(graph.keys()):
        topological_orders.append("".join(path))

    return topological_orders


# Testing the function
orders = find_all_topological_orders(graph)
print(f"Found {len(orders)} topological orderings:")
for order in orders:
    print(order)

Found 13 topological orderings:
abcdfe
abcfde
abcfed
abdcfe
acbdfe
acbfde
acbfed
acfbde
acfbed
bacdfe
bacfde
bacfed
badcfe
